In [7]:
import pandas as pd

# Set display options for pandas for easier printing
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 101)



sspm = pd.read_csv('data/stable_spm.csv')

player_data = pd.read_csv('data/stats_nba_player_data_2019-20.csv')


In [2]:
# print(sspm)
# print(player_data)

player_data = player_data[['TEAM_ABBREVIATION', 'AGE', 'GP', 'PLAYER_ID', 'PLAYER_NAME']]
player_data['AGE'] = player_data['AGE'].astype(int)

sspm = sspm[['Tm','Age', 'G', 'Player', 'Stable SPR', 'Positionally Adj Stable SPR']]
sspm.columns = ['TEAM_ABBREVIATION', 'AGE', 'GP', 'PLAYER_NAME', 'Stable SPR', 'Positionally Adj Stable SPR']


In [3]:
def fix_team(df, old, new):
    tm_idx = df[df['TEAM_ABBREVIATION'] == old].index
    df.loc[tm_idx, 'TEAM_ABBREVIATION'] = new
    return df

def fix_dups(group):
    if group.shape[0]==1:
        return group
    else:
        return group[group['TEAM_ABBREVIATION'] == 'TOT']

In [4]:
sspm = sspm.groupby(by='PLAYER_NAME').apply(fix_dups).reset_index(drop=True)

sspm=fix_team(sspm, 'PHO', 'PHX')
sspm=fix_team(sspm, 'BRK', 'BKN')
sspm=fix_team(sspm, 'CHO', 'CHA')

print(sspm)

    TEAM_ABBREVIATION  AGE  GP        PLAYER_NAME  Stable SPR  Positionally Adj Stable SPR
0                 ORL   24  29       Aaron Gordon        -0.4                         1.10
1                 IND   23  33      Aaron Holiday        -0.4                         0.84
2                 OKC   26  27        Abdel Nader        -1.7                        -0.34
3                 CHI   21   2        Adam Mokoka        -2.1                        -0.42
4                 WAS   22  20  Admiral Schofield        -1.6                         0.04
..                ...  ...  ..                ...         ...                          ...
472               MEM   25   4      Yuta Watanabe        -2.0                        -0.70
473               POR   22   3       Zach Collins        -1.3                         0.11
474               CHI   24  35        Zach LaVine         0.1                         1.83
475               LAL   22   2       Zach Norvell        -2.0                        -0.71

In [5]:
name_matches = player_data.merge(sspm, on='PLAYER_NAME', how='outer', suffixes=('_pd', '_sspm'))

name_matches_ids = name_matches.dropna()
name_matches_ids = name_matches_ids[['PLAYER_ID', 'PLAYER_NAME', 'Stable SPR', 'Positionally Adj Stable SPR']]

non_matches = name_matches[name_matches.isnull().any(axis=1)]

sspm_non_matches = non_matches[['TEAM_ABBREVIATION_sspm', 'AGE_sspm', 'GP_sspm', 'PLAYER_NAME', 'Stable SPR', 'Positionally Adj Stable SPR']].dropna()
sspm_non_matches.columns = ['TEAM_ABBREVIATION', 'AGE', 'GP', 'PLAYER_NAME', 'Stable SPR', 'Positionally Adj Stable SPR']
player_data_non_matches = non_matches[['TEAM_ABBREVIATION_pd', 'AGE_pd', 'GP_pd', 'PLAYER_ID', 'PLAYER_NAME']].dropna()
player_data_non_matches.columns = ['TEAM_ABBREVIATION', 'AGE', 'GP', 'PLAYER_ID', 'PLAYER_NAME']

In [6]:
joined = player_data_non_matches.merge(sspm_non_matches, how='outer', on=['TEAM_ABBREVIATION', 'AGE', 'GP'])
joined = joined.dropna()
joined = joined[['PLAYER_ID', 'PLAYER_NAME_x', 'Stable SPR', 'Positionally Adj Stable SPR']]
joined.columns = ['PLAYER_ID', 'PLAYER_NAME', 'Stable SPR', 'Positionally Adj Stable SPR']
prior = pd.concat([name_matches_ids, joined])
prior.to_csv('data/prior.csv', index=False)
print(prior)

     PLAYER_ID        PLAYER_NAME  Stable SPR  Positionally Adj Stable SPR
0     203932.0       Aaron Gordon        -0.4                         1.10
1    1628988.0      Aaron Holiday        -0.4                         0.84
2    1627846.0        Abdel Nader        -1.7                        -0.34
3    1629690.0        Adam Mokoka        -2.1                        -0.42
4    1629678.0  Admiral Schofield        -1.6                         0.04
..         ...                ...         ...                          ...
521  1627812.0       Yogi Ferrell        -2.3                        -0.65
522  1629139.0      Yuta Watanabe        -2.0                        -0.70
523  1628380.0       Zach Collins        -1.3                         0.11
524   203897.0        Zach LaVine         0.1                         1.83
528  1629597.0     Zylan Cheatham        -2.1                        -0.64

[414 rows x 4 columns]


In [8]:
sspm.head()

,Rk,Rk.1,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Adj FGA,Total FGA Est,Adj FGp,Adj FG,Total 3PA Est,Regressed 3P%,Total FTA,Adj FTp,Total 2PA,Adj 2Pp,GS%,Adj 3PA,Adj 2PA,Adj FTA,Adj ORB,Adj DRB,Adj AST,Adj STL,Adj BLK,Adj TO,Adj 3P,Adj 2P,Adj FT,Adj Sqrt(AST*TRB),3PA.1,2PA.1,FTA.1,ORB.1,DRB.1,AST.1,STL.1,BLK.1,TOV.1,3P.1,2P.1,FT.1,AST*TRB,Mean Regressed PTS100,Component Regressed SPR,Raw SPR,Minutes-Only Regressed SPR,Adj. Error,adj positional error,EMG,EM(1-G),MGS,MG(1-S),Positionally Adj Stable SPR,new contrib (ADJUSTMENT IS IN HERE),sum contrib tm,adjuster * mins,tm adjuster,Stable SPR,playertm,Equivalent End of Season Ranking
0,1,11,Giannis Antetokounmpo,PF,25,MIL,33,33,1022,16.7,30.1,0.556,2.5,7.6,0.331,14.2,22.5,0.631,9.0,15.0,0.599,3.9,15.3,19.2,8.3,1.8,1.7,5.3,4.7,44.9,117.0,96,30.1,640.879167,0.556,16.7356,161.816667,0.347,319.375000,0.607749,479.062500,0.600664,100%,7.5,22.2,14.3,3.4,12.2,7.1,1.6,1.5,4.1,2.6,13.3,8.7,10.538487,7.6,22.5,15.0,3.9,15.3,8.3,1.8,1.7,5.3,2.5,14.2,9.0,12.623787,43.2,8.21,9.73,8.42,-0.213868,-0.09,-218.573030,0.0,1022.0,0.0,8.30,4.891562,11.118645,-733.195643,-0.717413,7.6,Giannis AntetokounmpoMIL,1
1,2,182,James Harden,SG,30,HOU,33,33,1239,13.9,30.1,0.461,6.4,16.7,0.383,7.5,13.4,0.557,13.1,15.2,0.860,1.2,6.0,7.2,9.2,2.3,1.0,5.8,4.0,47.2,124.0,109,30.1,776.956250,0.461,13.8761,431.068750,0.373,392.350000,0.855129,345.887500,0.538337,100%,16.5,13.4,14.6,1.5,6.0,8.0,1.9,1.0,4.5,6.2,7.2,12.5,7.742388,16.7,13.4,15.2,1.2,6.0,9.2,2.3,1.0,5.8,6.4,7.5,13.1,8.138796,45.4,8.26,9.17,8.12,0.136550,0.06,169.185259,0.0,1239.0,0.0,8.20,6.183205,10.665072,-1738.941256,-1.403504,6.8,James HardenHOU,1
2,3,125,Luka Dončić,PG,20,DAL,30,30,972,13.9,29.3,0.472,4.5,13.8,0.325,9.4,15.6,0.602,11.1,13.8,0.809,2.2,12.1,14.3,13.2,1.8,0.2,6.3,3.5,43.3,121.0,106,29.3,593.325000,0.472,13.8296,279.450000,0.340,279.450000,0.805536,315.900000,0.568990,100%,13.6,15.5,13.2,2.3,10.0,10.7,1.6,0.5,4.5,4.6,8.8,10.6,11.444142,13.8,15.6,13.8,2.2,12.1,13.2,1.8,0.2,6.3,4.5,9.4,11.1,13.738996,42.2,7.76,8.54,7.33,0.434457,0.21,422.291764,0.0,972.0,0.0,7.55,4.470272,9.348545,-1033.942942,-1.063727,6.5,Luka DončićDAL,2
3,4,432,Karl-Anthony Towns,C,24,MIN,23,23,779,12.4,24.2,0.514,4.9,11.8,0.418,7.5,12.4,0.604,6.8,8.5,0.796,3.8,12.4,16.1,6.1,1.4,1.7,4.3,4.8,36.5,123.0,106,24.2,392.745833,0.514,12.4388,191.504167,0.383,137.947917,0.790990,201.241667,0.558767,100%,11.6,12.4,8.2,3.3,9.8,5.3,1.5,1.4,3.4,4.5,6.9,6.5,8.357926,11.8,12.4,8.5,3.8,12.4,6.1,1.4,1.7,4.3,4.9,7.5,6.8,9.940825,33.7,6.74,8.58,7.11,-0.373167,-0.20,-290.696979,0.0,779.0,0.0,6.93,3.240604,5.804002,-1131.200742,-1.452119,5.5,Karl-Anthony TownsMIN,4
4,5,114,Anthony Davis,PF,26,LAL,32,32,1127,13.0,25.9,0.502,1.4,5.0,0.288,11.6,20.9,0.553,9.7,11.3,0.853,3.2,9.5,12.7,4.5,1.9,3.4,3.3,3.4,37.1,120.0,101,25.9,608.110417,0.502,13.0018,117.395833,0.335,265.314583,0.846421,490.714583,0.539530,100%,5.0,20.7,10.9,3.0,8.4,4.3,1.7,2.7,3.0,1.7,11.2,9.2,6.971944,5.0,20.9,11.3,3.2,9.5,4.5,1.9,3.4,3.3,1.4,11.6,9.7,7.559762,36.6,6.36,7.32,6.40,-0.045940,-0.09,-51.774739,0.0,1127.0,0.0,6.44,4.435086,10.367869,-1440.888016,-1.278516,5.2,Anthony DavisLAL,4


In [9]:
player_data.head()

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK
0,203932,Aaron Gordon,Aaron,1610612753,ORL,24.0,62,30,32,0.484,2017.143333,335,767,0.437,73,237,0.308,151,224,0.674,107,368,475,228,100,51,39,42,125,205,894,-68,1976.0,20,1,1850.0,123,152,397,247,50,75,68,281,143,118,326,76,55,373,43,26,27,64,85,114,77,456,387,50,81,398,49,29,17,62
1,1628988,Aaron Holiday,Aaron,1610612754,IND,23.0,66,42,24,0.636,1617.331667,233,563,0.414,87,221,0.394,74,87,0.851,22,134,156,225,88,55,16,27,120,115,627,113,1276.7,3,0,1237.0,70,51,299,103,132,150,128,358,113,133,80,175,197,99,298,229,244,66,112,93,213,377,374,133,151,87,161,115,29,157
2,1627846,Abdel Nader,Abdel,1610612760,OKC,26.0,55,37,18,0.673,866.951667,123,263,0.468,48,128,0.375,51,66,0.773,14,86,100,38,43,23,20,10,78,49,345,-85,608.0,0,0,617.0,220,91,216,77,280,266,274,173,205,220,134,234,242,246,355,304,317,335,261,277,174,214,250,293,261,420,292,239,29,286
3,1629690,Adam Mokoka,Adam,1610612741,CHI,21.0,11,3,8,0.273,111.666667,12,28,0.429,6,15,0.400,2,4,0.500,7,3,10,4,2,4,0,4,17,4,32,50,60.0,0,0,60.0,441,462,103,473,444,448,453,306,362,391,69,468,462,464,412,499,463,467,481,428,480,121,101,460,451,128,456,239,29,455
4,1629678,Admiral Schofield,Admiral,1610612764,WAS,23.0,33,9,24,0.273,368.381667,35,92,0.380,19,61,0.311,10,15,0.667,7,40,47,15,7,8,4,4,50,18,99,-57,206.9,0,0,204.0,346,386,299,473,372,393,377,436,301,308,323,396,400,378,412,391,400,407,430,389,378,121,194,390,388,387,405,239,29,398
